In [1]:
pip install scikit-multilearn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
from skmultilearn.adapt import MLkNN 
from sklearn.metrics import hamming_loss, accuracy_score 

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [3]:
# DATA Exploration

In [4]:
# import data
df= pd.read_csv('Symptom2Disease.csv')
df.head()

,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1200 non-null   int64 
 1   label       1200 non-null   object
 2   text        1200 non-null   object
dtypes: int64(1), object(2)
memory usage: 28.3+ KB


In [6]:
df.drop(["Unnamed: 0"], axis = 1, inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1200 non-null   object
 1   text    1200 non-null   object
dtypes: object(2)
memory usage: 18.9+ KB


In [8]:
df.head()

,label,text
0,Psoriasis,I have been experiencing a skin rash on my arm...
1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,Psoriasis,I have been experiencing joint pain in my fing...
3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,Psoriasis,"My nails have small dents or pits in them, and..."


In [9]:
# Check for data classes
df['label'].nunique()

24

In [10]:
unique_values = df['text'].unique()
unique_count = df['text'].nunique()

#print("Unique values:", unique_values)
print("Count of unique values:", unique_count)

Count of unique values: 1153


In [11]:
a = df['label'].unique()
print(a)
print(type(a))

['Psoriasis' 'Varicose Veins' 'Typhoid' 'Chicken pox' 'Impetigo' 'Dengue'
 'Fungal infection' 'Common Cold' 'Pneumonia' 'Dimorphic Hemorrhoids'
 'Arthritis' 'Acne' 'Bronchial Asthma' 'Hypertension' 'Migraine'
 'Cervical spondylosis' 'Jaundice' 'Malaria' 'urinary tract infection'
 'allergy' 'gastroesophageal reflux disease' 'drug reaction'
 'peptic ulcer disease' 'diabetes']
<class 'numpy.ndarray'>


In [12]:
df.shape

(1200, 2)

In [13]:
df.duplicated().sum()

47

In [14]:
df[df.duplicated].head()

,label,text
163,Chicken pox,I'm feeling fatigued and have no energy. I can...
387,Common Cold,I've been quite exhausted and ill. My throat h...
430,Pneumonia,"I have a really high fever, and I have problem..."
433,Pneumonia,I'm having a hard time breathing and I feel re...
438,Pneumonia,"Lately I've been experiencing chills, fatigue,..."


In [15]:
df.drop_duplicates(inplace= True)

In [16]:
df.shape

(1153, 2)

In [17]:
df['label'].value_counts()

label
Psoriasis                          50
Varicose Veins                     50
peptic ulcer disease               50
drug reaction                      50
allergy                            50
urinary tract infection            50
Hypertension                       50
diabetes                           50
Fungal infection                   50
Dengue                             50
Impetigo                           50
Typhoid                            50
Common Cold                        49
Cervical spondylosis               49
Chicken pox                        49
Bronchial Asthma                   49
gastroesophageal reflux disease    48
Pneumonia                          47
Migraine                           47
Arthritis                          46
Acne                               46
Malaria                            44
Dimorphic Hemorrhoids              41
Jaundice                           38
Name: count, dtype: int64

In [18]:
df.head()

,label,text
0,Psoriasis,I have been experiencing a skin rash on my arm...
1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,Psoriasis,I have been experiencing joint pain in my fing...
3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,Psoriasis,"My nails have small dents or pits in them, and..."


In [19]:
df.tail()

,label,text
1195,diabetes,I'm shaking and trembling all over. I've lost ...
1196,diabetes,"Particularly in the crevices of my skin, I hav..."
1197,diabetes,I regularly experience these intense urges and...
1198,diabetes,"I have trouble breathing, especially outside. ..."
1199,diabetes,I constantly sneeze and have a dry cough. My i...


In [20]:
df.isnull().sum() # to check null value

label    0
text     0
dtype: int64

In [21]:
#df.drop('label',axis=1, inplace=True)

#### One hot Encoding

In [188]:
#Extract categorical columns from the dataframe
categorical_columns = ['label']
#Initialize OneHotEncoder
encoder = OneHotEncoder()

# Apply one-hot encoding to the categorical columns
one_hot_encoded = encoder.fit_transform(df[['label']])


In [189]:
# Convert sparse matrix to a dense array
one_hot_encoded_array = one_hot_encoded.toarray()
# # Create new column names based on the categories
column_names = encoder.get_feature_names_out(['label'])
# # Create a DataFrame from the one-hot encoded array
one_hot_encoded_df = pd.DataFrame(one_hot_encoded_array, columns=column_names)



In [194]:
one_hot_encoded_df.shape

(1153, 24)

## Data Preprocessing

In [191]:
df['text']

0       I have been experiencing a skin rash on my arm...
1       My skin has been peeling, especially on my kne...
2       I have been experiencing joint pain in my fing...
3       There is a silver like dusting on my skin, esp...
4       My nails have small dents or pits in them, and...
                              ...                        
1195    I'm shaking and trembling all over. I've lost ...
1196    Particularly in the crevices of my skin, I hav...
1197    I regularly experience these intense urges and...
1198    I have trouble breathing, especially outside. ...
1199    I constantly sneeze and have a dry cough. My i...
Name: text, Length: 1153, dtype: object

In [197]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings
data = df
if not sys.warnoptions:
    warnings.simplefilter("ignore")
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent
data['text'] = data['text'].str.lower()
data['text'] = data['text'].apply(cleanHtml)
data['text'] = data['text'].apply(cleanPunc)
data['text'] = data['text'].apply(keepAlpha)

#Stop word Removal

stop_words = set(stopwords.words('english'))
stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)
data['text'] = data['text'].apply(removeStopWords)


In [199]:
#stemming
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence
data['text'] = data['text'].apply(stemming)

In [200]:
data

,label,text
0,Psoriasis,experienc skin rash arm leg torso past week re...
1,Psoriasis,skin peel especi knee elbow scalp peel often a...
2,Psoriasis,experienc joint pain finger wrist knee pain of...
3,Psoriasis,silver like dust skin especi lower back scalp ...
4,Psoriasis,nail small dent pit often feel inflammatori te...
...,...,...
1195,diabetes,im shake trembl ive lost sens tast smell im ex...
1196,diabetes,particular crevic skin skin rash irrit skin br...
1197,diabetes,regular experi intens urg want urin frequent f...
1198,diabetes,troubl breath especi outsid start feel hot sta...


In [201]:
data['text'].isnull().sum() # to check null value

0

In [202]:
text_df = data.drop('label', axis=1)

In [203]:
text_df

,text
0,experienc skin rash arm leg torso past week re...
1,skin peel especi knee elbow scalp peel often a...
2,experienc joint pain finger wrist knee pain of...
3,silver like dust skin especi lower back scalp ...
4,nail small dent pit often feel inflammatori te...
...,...
1195,im shake trembl ive lost sens tast smell im ex...
1196,particular crevic skin skin rash irrit skin br...
1197,regular experi intens urg want urin frequent f...
1198,troubl breath especi outsid start feel hot sta...


In [204]:
text_df.isnull().sum() # to check null value

text    0
dtype: int64

In [205]:
# Concatenate the one-hot encoded DataFrame with the original DataFrame
df_with_one_hot = pd.concat([text_df, one_hot_encoded_df], axis=1)


In [206]:

df_with_one_hot

,text,label_Acne,label_Arthritis,label_Bronchial Asthma,label_Cervical spondylosis,label_Chicken pox,label_Common Cold,label_Dengue,label_Dimorphic Hemorrhoids,label_Fungal infection,...,label_Pneumonia,label_Psoriasis,label_Typhoid,label_Varicose Veins,label_allergy,label_diabetes,label_drug reaction,label_gastroesophageal reflux disease,label_peptic ulcer disease,label_urinary tract infection
0,experienc skin rash arm leg torso past week re...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,skin peel especi knee elbow scalp peel often a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,experienc joint pain finger wrist knee pain of...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,silver like dust skin especi lower back scalp ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,nail small dent pit often feel inflammatori te...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
873,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
894,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1048,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [137]:
df_with_one_hot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1200 entries, 0 to 1049
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   text                                   1153 non-null   object 
 1   label_Acne                             1153 non-null   float64
 2   label_Arthritis                        1153 non-null   float64
 3   label_Bronchial Asthma                 1153 non-null   float64
 4   label_Cervical spondylosis             1153 non-null   float64
 5   label_Chicken pox                      1153 non-null   float64
 6   label_Common Cold                      1153 non-null   float64
 7   label_Dengue                           1153 non-null   float64
 8   label_Dimorphic Hemorrhoids            1153 non-null   float64
 9   label_Fungal infection                 1153 non-null   float64
 10  label_Hypertension                     1153 non-null   float64
 11  label_Imp

In [209]:
df_with_one_hot['text'].isnull().sum() # to check null value

47

In [210]:
df_with_one_hot.isnull().sum() # to check null value


text                                     47
label_Acne                               47
label_Arthritis                          47
label_Bronchial Asthma                   47
label_Cervical spondylosis               47
label_Chicken pox                        47
label_Common Cold                        47
label_Dengue                             47
label_Dimorphic Hemorrhoids              47
label_Fungal infection                   47
label_Hypertension                       47
label_Impetigo                           47
label_Jaundice                           47
label_Malaria                            47
label_Migraine                           47
label_Pneumonia                          47
label_Psoriasis                          47
label_Typhoid                            47
label_Varicose Veins                     47
label_allergy                            47
label_diabetes                           47
label_drug reaction                      47
label_gastroesophageal reflux di

In [213]:
df_cleaned = df_with_one_hot.dropna()


In [214]:
df_cleaned.isnull().sum() # to check null value

text                                     0
label_Acne                               0
label_Arthritis                          0
label_Bronchial Asthma                   0
label_Cervical spondylosis               0
label_Chicken pox                        0
label_Common Cold                        0
label_Dengue                             0
label_Dimorphic Hemorrhoids              0
label_Fungal infection                   0
label_Hypertension                       0
label_Impetigo                           0
label_Jaundice                           0
label_Malaria                            0
label_Migraine                           0
label_Pneumonia                          0
label_Psoriasis                          0
label_Typhoid                            0
label_Varicose Veins                     0
label_allergy                            0
label_diabetes                           0
label_drug reaction                      0
label_gastroesophageal reflux disease    0
label_pepti

In [215]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1106 entries, 0 to 1152
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   text                                   1106 non-null   object 
 1   label_Acne                             1106 non-null   float64
 2   label_Arthritis                        1106 non-null   float64
 3   label_Bronchial Asthma                 1106 non-null   float64
 4   label_Cervical spondylosis             1106 non-null   float64
 5   label_Chicken pox                      1106 non-null   float64
 6   label_Common Cold                      1106 non-null   float64
 7   label_Dengue                           1106 non-null   float64
 8   label_Dimorphic Hemorrhoids            1106 non-null   float64
 9   label_Fungal infection                 1106 non-null   float64
 10  label_Hypertension                     1106 non-null   float64
 11  label_Imp

In [216]:
df_cleaned.head()

,text,label_Acne,label_Arthritis,label_Bronchial Asthma,label_Cervical spondylosis,label_Chicken pox,label_Common Cold,label_Dengue,label_Dimorphic Hemorrhoids,label_Fungal infection,...,label_Pneumonia,label_Psoriasis,label_Typhoid,label_Varicose Veins,label_allergy,label_diabetes,label_drug reaction,label_gastroesophageal reflux disease,label_peptic ulcer disease,label_urinary tract infection
0,experienc skin rash arm leg torso past week re...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,skin peel especi knee elbow scalp peel often a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,experienc joint pain finger wrist knee pain of...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,silver like dust skin especi lower back scalp ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,nail small dent pit often feel inflammatori te...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [217]:
df_cleaned.tail()

,text,label_Acne,label_Arthritis,label_Bronchial Asthma,label_Cervical spondylosis,label_Chicken pox,label_Common Cold,label_Dengue,label_Dimorphic Hemorrhoids,label_Fungal infection,...,label_Pneumonia,label_Psoriasis,label_Typhoid,label_Varicose Veins,label_allergy,label_diabetes,label_drug reaction,label_gastroesophageal reflux disease,label_peptic ulcer disease,label_urinary tract infection
1148,mouth fill sour acid flavour frequent belch bu...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1149,unintent lose weight find challeng gain back r...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1150,increas thirst frequent urin often dri mouth t...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1151,blur vision seem get wors feel fatigu tire tim...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1152,dri mouth throat experienc increas appetit hun...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [152]:
df_cleaned.shape

(1106, 25)

## Formatdf_cleaned Training

In [226]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [227]:
train, test = train_test_split(df_cleaned, random_state=42, test_size=0.10, shuffle=True)



In [228]:
print(train.shape)
print(test.shape)

(995, 25)
(111, 25)


In [255]:
train

,text,label_Acne,label_Arthritis,label_Bronchial Asthma,label_Cervical spondylosis,label_Chicken pox,label_Common Cold,label_Dengue,label_Dimorphic Hemorrhoids,label_Fungal infection,...,label_Pneumonia,label_Psoriasis,label_Typhoid,label_Varicose Veins,label_allergy,label_diabetes,label_drug reaction,label_gastroesophageal reflux disease,label_peptic ulcer disease,label_urinary tract infection
23,im joint discomfort finger wrist knee pain fre...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
221,sore face most near nose lip sore caus discomf...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
378,ive cough lot feel chilli shiveri nose quit cl...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
550,ive realli bad rash skin late full pus fill pi...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
564,skin simpli nonherit nasti rash pack pus fill ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,im lot troubl go bathroom realli pain stool bl...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121,im sever stomach pain constip everi night get ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1091,month cycl chang ive unexpect vagin discharg o...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1142,regular troubl swallow sometim feel like food ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [229]:
train_text = train['text']
test_text = test['text']

In [253]:
train_text.shape

(995,)

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(train_text)
vectorizer.fit(test_text)

In [230]:
vectorizer

TfidfVectorizer(ngram_range=(1, 3), strip_accents='unicode')

In [260]:
x_train = vectorizer.transform(train['text'])
y_train = train.drop(labels = ['text'], axis=1)

x_test = vectorizer.transform(test['text'])
y_test = test.drop(labels = ['text'], axis=1)

In [261]:
x_train

<995x54 sparse matrix of type '<class 'numpy.float64'>'
	with 45 stored elements in Compressed Sparse Row format>

In [275]:
x_train.todense()[1]


matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])

### Multiple Binary Classifications - (Binary Relevance)


In [270]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())

# train
classifier.fit(x_train, y_train)



BinaryRelevance(classifier=GaussianNB(), require_dense=[True, True])

In [271]:
x_test

<111x54 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [272]:
# predict
predictions = classifier.predict(x_test)
predictions


<111x24 sparse matrix of type '<class 'numpy.float64'>'
	with 2495 stored elements in Compressed Sparse Column format>

In [273]:
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.0


